# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [4]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)



In [25]:
def get_score(value, ranges):
    
    """
    Get the score from a given range of values.
    
    
    Args:
        value: `int` 
            value to find the score for
        ranges: `list of dict`
            List of dictionary with the `min` and` max` and points for each ranges.
            e.g., 
            [
            {"min":  7.70, "max": 99999,  "points": 4},
            {"min":  7.60, "max": 7.70,  "points": 3}
            ]
        
    Return: `int`
        Points for a given value. 
    
    
    """
    
    for _range in ranges:
        if value >= _range['min'] and value < _range['max']:
            return _range['points']
    

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [30]:
assert get_score(38, rules['Temperature'])==0
assert get_score(24, rules['Respiratory Rate'])==0
assert get_score(2, rules['Potassium'])==4

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


In [146]:
import json
def apache_score(organ_failure, age, creatinine, temperature, ph, heart_rate, 
                  respiratory_rate, sodium, potassium, hematocrit, white_blood_count, 
                  fio2, glasgow_value, pao2, aa_gradient, acute_renal_failure,  config_file):
    
    """
    This funcion will compute the apache score.
    
    Args:
        organ_failure: `str`
            Organ failure history. It can be one of the following values
            "Nonoperative",  "Emergency", "Elective", "None" 
        age: `int` 
            Age of the patient
        creatinine
        temperature
        ph
        heart_rate
        respiratory_rate
        , sodium, potassium, hematocrit, white_blood_count, 
                  fio2, glasgow_value, pao2, aa_gradient,  
                  
        acute_renal_failure: `str`
            There are two possible values.
                1. Acute Renal Failure
                2. Chronic Renal Failure
                  
        config_file: `str`
            Path for the apache score configuration. DESCRIBE HERE THE FILE.
                  
    Return: `int`
        Apache score.
        
    
    """
    
    range_loop_params = [
        'Age',
        'Temperature',
        'pH',
        'Heart Rate',
        'Respiratory Rate',
        'Sodium',
        'Potassium',
        'Hematocrit',
        'White Blood Count',
    ]
    
    value_dict = {
        'Organ Failure History': organ_failure,
        'Age':age,
        'Temperature': temperature,
        'pH': ph,
        'Heart Rate': heart_rate,
        'Respiratory Rate': respiratory_rate,
        'Sodium': sodium,
        'Potassium': potassium,
        'Hematocrit': hematocrit,
        'White Blood Count': white_blood_count,
        'Creatinine': creatinine,
        'FiO2': fio2,
        'A-a gradient' : aa_gradient,
        'PaO2': pao2,
    }

    
    score = 15 - glasgow_value
    config = json.load(open(config_file))
    for param in range_loop_params:
        curr_value = value_dict[param]
        list_of_ranges = config[param]
        print(get_score(curr_value, list_of_ranges), 'for ->', param)
        score += get_score(curr_value, list_of_ranges)
        
    
    #Creatinene and FiO2 cases.
    
    if fio2 <50:
        curr_value = value_dict['PaO2']
        list_of_ranges = config['FiO2']['PaO2']
        print(get_score(curr_value, list_of_ranges), 'for ->', 'Pao2')
        score += get_score(curr_value, list_of_ranges)
    else:
        curr_value = value_dict['A-a gradient']
        list_of_ranges = config['FiO2']['A-a gradient']
        print(get_score(curr_value, list_of_ranges), 'for ->', 'aa grad')
        score += get_score(curr_value, list_of_ranges)
        

    curr_value = value_dict['Creatinine']
    if acute_renal_failure=='Acute Renal Failure':
        list_of_ranges = config['Creatinine']['Acute Renal Failure']
        print(get_score(curr_value, list_of_ranges), 'for ->', 'acute')
        score += get_score(curr_value, list_of_ranges)
    elif acute_renal_failure=='Chronic Renal Failure':
        list_of_ranges = config['Creatinine']['Chronic Renal Failure']
        print(get_score(curr_value, list_of_ranges), 'for ->', 'renal')
        score += get_score(curr_value, list_of_ranges)
    else:
        raise ValueError
    
    #Score for organ failure
    print(config['Organ Failure History'][organ_failure], ' for ->', organ_failure)
    score += config['Organ Failure History'][organ_failure]
    
    return score

### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

In [147]:
assert apache_score(organ_failure='None', age=44, creatinine=0.7, temperature=37, ph=8, heart_rate=44, 
                  respiratory_rate=13, sodium=131, potassium=4, hematocrit=31, white_blood_count=4, 
                  fio2=51, glasgow_value=14, pao2=11, aa_gradient=210, acute_renal_failure='Acute Renal Failure',config_file='apache.json')==10

0 for -> Age
0 for -> Temperature
4 for -> pH
3 for -> Heart Rate
0 for -> Respiratory Rate
0 for -> Sodium
0 for -> Potassium
0 for -> Hematocrit
0 for -> White Blood Count
2 for -> aa grad
0 for -> acute
0  for -> None


In [148]:
assert apache_score(organ_failure='Nonoperative', age=55, creatinine=1.6, temperature=33, ph=7.17, heart_rate=44, 
                  respiratory_rate=13, sodium=131, potassium=4, hematocrit=31, white_blood_count=4, 
                  fio2=51, glasgow_value=14, pao2=11, aa_gradient=210, acute_renal_failure='Acute Renal Failure', config_file='apache.json')==23

3 for -> Age
2 for -> Temperature
3 for -> pH
3 for -> Heart Rate
0 for -> Respiratory Rate
0 for -> Sodium
0 for -> Potassium
0 for -> Hematocrit
0 for -> White Blood Count
2 for -> aa grad
4 for -> acute
5  for -> Nonoperative


---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [139]:
def read_patient_csv(url, sep=','):
    
    value_dict = {
        'Organ Failure History': 'organ_failure',
        'Age':'age',
        'Temperature': 'temperature',
        'pH': 'ph',
        'Heart Rate': 'heart_rate',
        'Respiratory Rate': 'respiratory_rate',
        'Sodium': 'sodium',
        'Potassium': 'potassium',
        'Hematocrit': 'hematocrit',
        'White Blood Count': 'white_blood_count',
        'Creatinine': 'creatinine',
        'FiO2': 'fio2',
        'A-a Gradient' : 'aa_gradient',
        'PaO2': 'pao2',
        'Acute Renal Failure': 'acute_renal_failure',
    'Glasgow Coma Scale':'glasgow_value'
    }
    
    
    df = pd.read_csv(url, sep=sep)
    
    df = df.rename(columns=value_dict)
    
    scores = []
    for row, patient in df.drop(columns=['Patient']).iterrows():
        patient = patient.to_dict()
        scores.append(apache_score(**patient, config_file='apache.json'))
        print(patient)
                                
    return scores

### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv




In [ ]:
import pandas as pd
df = read_patient_csv("https://hds5210-2020.s3.amazonaws.com/TestPatients.csv")

In [130]:
df_p = pd.read_csv('https://hds5210-2020.s3.amazonaws.com/TestPatients.csv')

In [141]:
df_test = pd.read_csv('https://hds5210-2020.s3.amazonaws.com/Scores.csv')

In [142]:
df_test['ours'] = df

In [143]:
(df_test['TOTAL']==df_test['ours']).sum()

1

In [145]:
df_test

,TOTAL,ours
0,46,35
1,38,31
2,46,47
3,37,34
4,33,44
...,...,...
350,41,46
351,32,45
352,36,43
353,57,44
